<a href="https://colab.research.google.com/github/anshupandey/Generative-AI-for-Professionals/blob/main/EY2024/C7_parallel_function_calling_with_Azure_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with Parallel Function Calls and Multiple Function Responses in Gemini

## Overview

Gemini is a family of generative AI models developed by Google DeepMind that is designed for multimodal use cases. The Gemini API gives you access to the Gemini Pro and Gemini Pro Vision models.

[Function Calling](https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling) in Gemini lets you create a description of a function in your code, then pass that description to a language model in a request. The response from the model includes the name of a function that matches the description and the arguments to call it with.

In this tutorial, you'll learn how to work with parallel function calling within Gemini Function Calling, including:
    
- Handling parallel function calls for repeated functions
- Working with parallel function calls across multiple functions
- Extracting multiple function calls from a Gemini response
- Calling multiple functions and returning them to Gemini

### What is parallel function calling?

In previous versions of the Gemini Pro models (prior to May 2024), Gemini would return two or more chained function calls if the model determined that more than one function call was needed before returning a natural language summary. Here, a chained function call means that you get the first function call response, return the API data to Gemini, get a second function call response, return the API data to Gemini, and so on.

In recent versions of specific Gemini Pro models (from May 2024 and on), Gemini has the ability to return two or more function calls in parallel (i.e., two or more function call responses within the first function call response object). Parallel function calling allows you to fan out and parallelize your API calls or other actions that you perform in your application code, so you don't have to work through each function call response and return one-by-one! Refer to the [Gemini Function Calling documentation](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling) for more information on which Gemini model versions support parallel function calling.


<img src="https://storage.googleapis.com/github-repo/generative-ai/gemini/function-calling/parallel-function-calling-in-gemini.png">

In [1]:
!pip install openai wikipedia --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.2 MB/s eta 0:00:00


In [2]:
api_key = "62d77bb382974118aec75d84d274bb72"
api_version = "2024-02-01" # "2023-05-15"
azure_endpoint = "https://eygpt24.openai.azure.com/"
deployment_name = "gpt-4o"

In [3]:
from openai import AzureOpenAI

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=azure_endpoint,
    api_key = api_key,)

### Define helper function

In [97]:
def extract_tool_calls(response: dict):
    tool_calls = []
    tool_call = response.choices[0].message.tool_calls
    if tool_call:
        for tool in tool_call:
            function_name = tool.function.name
            function_args = eval(tool.function.arguments)  # Safely convert string to dictionary
            function_call_dict = {function_name: function_args}
            function_call_dict['id'] = tool.id
            tool_calls.append(function_call_dict)

    return tool_calls

## Example: Parallel function calls on the same function

A great use case for parallel function calling is when you have a function that only accepts one parameter per API call and you need to make repeated calls to that function.

With Parallel Function Calling, rather than having to send N number of API requests to LLM for N number function calls, instead you can send a single API request to LLM, receive N number of Function Call Responses within a single response, make N number of external API calls in your code, then return all of the API responses to LLM in bulk. And you can do all of this without any extra configuration in your function declarations, tools, or requests to LLM.

In this example, you'll do exactly that and use Parallel Function Calling in LLM to ask about multiple topics on [Wikipedia](https://www.wikipedia.org/). Let's get started!


In [90]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_wikipedia",
            "description": "Search for articles or any other relevant information on Wikipedia'",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Query to search for on Wikipedia"
                    }
                },
                "required": ["query"],
                "additionalProperties": False
            }
        }
    }
]

In [98]:
### Send prompt to OpenAI GPT-4o with function calling
prompt = "Search for articles related to solar panels, renewable energy, and battery storage and provide a summary of your findings"

In [99]:
response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    tools=tools,
    tool_choice="auto",
)


In [112]:
response.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_45ZnM7TX4U2b6wrg7BsL2hbg', function=Function(arguments='{"topic": "solar panels"}', name='summarize_wikipedia'), type='function'),
 ChatCompletionMessageToolCall(id='call_Cxp2KRWsiYbI7k3ZBiivJg4C', function=Function(arguments='{"topic": "renewable energy"}', name='summarize_wikipedia'), type='function'),
 ChatCompletionMessageToolCall(id='call_1XY4OgwZIeevaUsqxf7sBbK3', function=Function(arguments='{"topic": "battery storage"}', name='summarize_wikipedia'), type='function')]

In [101]:
print(response.model_dump_json(indent=2))

{
  "id": "chatcmpl-A8IeEmTFX8hew9fbOxuARvobS0vri",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "function_call": null,
        "tool_calls": [
          {
            "id": "call_nYpjZeCj3s7Wxn6c5QkCY0Lu",
            "function": {
              "arguments": "{\"query\": \"solar panels\"}",
              "name": "search_wikipedia"
            },
            "type": "function"
          },
          {
            "id": "call_r9qHvI2OhbavkEJvrc7hUTNQ",
            "function": {
              "arguments": "{\"query\": \"renewable energy\"}",
              "name": "search_wikipedia"
            },
            "type": "function"
          },
          {
            "id": "call_Q5j6bUoVmxjb9GvI5TMlKlcz",
            "function": {
              "arguments": "{\"query\": \"battery storage\"}",
              "name": "search_wikipedia"


In [102]:
# Extract function calls from the response
tool_calls = extract_tool_calls(response)
print(tool_calls)

[{'search_wikipedia': {'query': 'solar panels'}, 'id': 'call_nYpjZeCj3s7Wxn6c5QkCY0Lu'}, {'search_wikipedia': {'query': 'renewable energy'}, 'id': 'call_r9qHvI2OhbavkEJvrc7hUTNQ'}, {'search_wikipedia': {'query': 'battery storage'}, 'id': 'call_Q5j6bUoVmxjb9GvI5TMlKlcz'}]


In [103]:
!pip install wikipedia --quiet

In [104]:
### Make external API calls
import wikipedia
api_response = []

# Loop over multiple function calls
for tool in tool_calls:
    print(tool)

    # Make external API call
    result = wikipedia.summary(tool["search_wikipedia"]["query"])

    # Collect all API responses
    api_response.append(result)

{'search_wikipedia': {'query': 'solar panels'}, 'id': 'call_nYpjZeCj3s7Wxn6c5QkCY0Lu'}
{'search_wikipedia': {'query': 'renewable energy'}, 'id': 'call_r9qHvI2OhbavkEJvrc7hUTNQ'}
{'search_wikipedia': {'query': 'battery storage'}, 'id': 'call_Q5j6bUoVmxjb9GvI5TMlKlcz'}


In [105]:
for res in api_response:
    print(res)

A solar panel is a device that converts sunlight into electricity by using photovoltaic (PV) cells. PV cells are made of materials that produce excited electrons when exposed to light. The electrons flow through a circuit and produce direct current (DC) electricity, which can be used to power various devices or be stored in batteries. Solar panels are also known as solar cell panels, solar electric panels, or PV modules.
Solar panels are usually arranged in groups called arrays or systems. A photovoltaic system consists of one or more solar panels, an inverter that converts DC electricity to alternating current (AC) electricity, and sometimes other components such as controllers, meters, and trackers. Most panels are in solar farms, which  supply the electricity grid as can some rooftop solar.
Some advantages of solar panels are that they use a renewable and clean source of energy, reduce greenhouse gas emissions, and lower electricity bills. Some disadvantages are that they depend on 

In [106]:

### Generate a natural language summary
messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},]

messages.append(response.choices[0].message)
for tool,res in zip(tool_calls,api_response):
  messages.append({"tool_call_id": tool['id'],
                   "role": "tool",
                   "name": 'search_wikipedia',
                   "content": res,})


In [107]:
# Now we return all API responses back to GPT-4 to summarize

new_response = client.chat.completions.create(
    model="gpt-4o",
    messages = messages)

In [108]:

# Display the final summary
print(new_response.choices[0].message.content)

Here's a summary of key findings from the articles on solar panels, renewable energy, and battery storage:

### Solar Panels
- **Function**: Solar panels convert sunlight into electricity using photovoltaic (PV) cells.
- **Components**: PV cells, inverters, and sometimes controllers, meters, and trackers.
- **Use Cases**: Residential, commercial, industrial, and space applications.
- **Advantages**: Renewable, clean energy source, reduces greenhouse gas emissions, lowers electricity bills.
- **Disadvantages**: Dependent on sunlight, requires maintenance, high initial costs.
- **Deployments**: Solar farms and rooftop systems contribute to grid electricity.

### Renewable Energy
- **Types**: Solar energy, wind power, hydropower, bioenergy, and geothermal power.
- **Benefits**: Reduces greenhouse gas emissions, less air pollution, supports electrification, more cost-effective and efficient over the last 30 years.
- **Current and Future Trends**: Significant growth, accounting for 30% of g

## Example: Parallel function calls across multiple functions

Another good fit for parallel function calling is when you have multiple, independent functions that you want to be able to call in parallel, which reduces the number of consecutive OpenAI API calls that you need to make and (ideally) reduces the overall response time to the end user who is waiting for a natural language response.

In this example, you'll use Parallel Function Calling in LLM to ask about multiple aspects of topics and articles on [Wikipedia](https://www.wikipedia.org/).


In [109]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_wikipedia",
            "description": "Search for articles or any other relevant information on Wikipedia",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Query to search for on Wikipedia"
                    }
                },
                "required": ["query"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "suggest_wikipedia",
            "description": "Get suggested titles from Wikipedia based on the given query term",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Query to search for suggested titles on Wikipedia"
                    }
                },
                "required": ["query"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "summarize_wikipedia",
            "description": "Retrieve brief summaries of Wikipedia articles related to a given topic",
            "parameters": {
                "type": "object",
                "properties": {
                    "topic": {
                        "type": "string",
                        "description": "Topic to search for article summaries on Wikipedia"
                    }
                },
                "required": ["topic"],
                "additionalProperties": False
            }
        }
    }
]


In [114]:
prompt = "Show the search results, variations, and article summaries about Wikipedia articles related to the solar system"

In [115]:
response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    tools=tools,
    tool_choice='auto'
)


In [116]:
response.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_5PP5ka4a6YAnemaziAvX8lGy', function=Function(arguments='{"query": "solar system"}', name='search_wikipedia'), type='function'),
 ChatCompletionMessageToolCall(id='call_GLjVa0dK1m8fGDJV4FCmaVTq', function=Function(arguments='{"query": "solar system"}', name='suggest_wikipedia'), type='function'),
 ChatCompletionMessageToolCall(id='call_43xKeAMpuUCdL2gTkIBavR6n', function=Function(arguments='{"topic": "solar system"}', name='summarize_wikipedia'), type='function')]

### Extract function names and parameters

Use the helper function that we created earlier to extract the function names and function parameters for each Function Call that LLM responded with:

In [117]:
# Extract function calls from the response
tool_calls = extract_tool_calls(response)
print(tool_calls)

[{'search_wikipedia': {'query': 'solar system'}, 'id': 'call_5PP5ka4a6YAnemaziAvX8lGy'}, {'suggest_wikipedia': {'query': 'solar system'}, 'id': 'call_GLjVa0dK1m8fGDJV4FCmaVTq'}, {'summarize_wikipedia': {'topic': 'solar system'}, 'id': 'call_43xKeAMpuUCdL2gTkIBavR6n'}]


### Make external API calls

Next, you'll loop through the Function Calls and use the `wikipedia` Python package to make APIs calls and gather information from Wikipedia:

In [121]:
api_response = {}

# Loop over multiple function calls
for tool in tool_calls:
    # Extract the function name
    print(tool)
    function_name = list(tool.keys())[0]

    # Determine which external API call to make
    if function_name == "search_wikipedia":
        result = wikipedia.search(tool["search_wikipedia"]["query"])
    if function_name == "suggest_wikipedia":
        result = wikipedia.suggest(tool["suggest_wikipedia"]["query"])
    if function_name == "summarize_wikipedia":
        result = wikipedia.summary(
            tool["summarize_wikipedia"]["topic"], auto_suggest=False
        )

    # Collect all API responses
    api_response[function_name] = result

{'search_wikipedia': {'query': 'solar system'}, 'id': 'call_5PP5ka4a6YAnemaziAvX8lGy'}
{'suggest_wikipedia': {'query': 'solar system'}, 'id': 'call_GLjVa0dK1m8fGDJV4FCmaVTq'}
{'summarize_wikipedia': {'topic': 'solar system'}, 'id': 'call_43xKeAMpuUCdL2gTkIBavR6n'}


In [122]:
api_response

{'search_wikipedia': ['Solar System',
  'Formation and evolution of the Solar System',
  'List of Solar System objects by size',
  'Solar System (disambiguation)',
  'List of Solar System objects',
  'Photovoltaic system',
  'Passive solar building design',
  'Solar System belts',
  'List of gravitationally rounded objects of the Solar System',
  'Solar System model'],
 'suggest_wikipedia': 'soler system',
 'summarize_wikipedia': "The Solar System is the gravitationally bound system of the Sun and the objects that orbit it. It was formed about 4.6 billion years ago when a dense region of a molecular cloud collapsed, forming the Sun and a protoplanetary disc. The Sun is a typical star that maintains a balanced equilibrium by the fusion of hydrogen into helium at its core, releasing this energy from its outer photosphere. Astronomers classify it as a G-type main-sequence star.\nThe largest objects that orbit the Sun are the eight planets. In order from the Sun, they are four terrestrial 

### Get a natural language summary

Now you can return all of the API responses to LLM so that it can generate a natural language summary:

In [125]:

### Generate a natural language summary
messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},]

messages.append(response.choices[0].message)
for tool,res in zip(tool_calls,api_response):
  function_name = list(tool.keys())[0]
  messages.append({"tool_call_id": tool['id'],
                   "role": "tool",
                   "name": function_name,
                   "content": res,})


In [126]:
# Now we return all API responses back to GPT-4 to summarize

new_response = client.chat.completions.create(
    model="gpt-4o",
    messages = messages)

In [127]:

# Display the final summary
print(new_response.choices[0].message.content)

### Search Results for "solar system":
1. **Solar System**
   - Overview of the Solar System including all its planets, moons, and other celestial objects.
2. **Formation and evolution of the Solar System**
   - Detailed explanation about how the Solar System was formed and its evolutionary processes.
3. **List of Solar System objects**
   - Comprehensive list of all objects found within the Solar System, including minor planets, moons, and comets.
4. **Exploration of the Solar System**
   - History and future plans regarding the exploration of the Solar System by various space missions.
5. **Solar System model**
   - Models and educational representations used to illustrate the components and scale of the Solar System.

### Variations and Related Topics:
1. **Astronomy**
   - The study of celestial objects, space, and the universe as a whole.
2. **Planetary science**
   - The scientific study of planets, including those in our Solar System and beyond.
3. **Cosmology**
   - The science